In [1]:
import pandas as pd

In [2]:
# en_10k.txt와 ko_5.8k.xlsx를 읽어와서 각각 en_df, ko_df에 저장
en_df = pd.read_csv('en_10k.txt', header=None)
ko_df = pd.read_excel('ko_5.8k.xlsx')

display(en_df.head())
display(ko_df.head())

,0
0,the
1,of
2,and
3,to
4,a


,순위,단어,품사,풀이,등급
0,1.0,것01,의,NaN,A
1,2.0,하다01,동,NaN,A
2,3.0,있다01,보,NaN,A
3,4.0,있다01,형,NaN,A
4,5.0,되다01,동,어른이 ~,A


In [3]:
used_ko_part = ['고', '대', '동', '명', '부', '형']
print(len(ko_df))
ko_df = ko_df[ko_df['품사'].isin(used_ko_part)]
print(len(ko_df))

5965
5628


In [4]:
# ko_df의 '단어' 칼럼의 텍스트 내 숫자가 들어가는 경우 숫자를 제거
ko_df['단어'] = ko_df['단어'].str.replace('\d+', '', regex=True)
display(ko_df.head())

,순위,단어,품사,풀이,등급
1,2.0,하다,동,NaN,A
3,4.0,있다,형,NaN,A
4,5.0,되다,동,어른이 ~,A
7,8.0,나,대,NaN,A
9,10.0,없다,형,NaN,A


In [7]:
ko_df = ko_df[['단어']]
ko_df.drop_duplicates(inplace=True)
ko_df = ko_df.reset_index(drop=True)
display(ko_df.head())
print(len(ko_df))

,단어
0,하다
1,있다
2,되다
3,나
4,없다


5351


In [8]:
ko_df.columns = ['ko']
en_df.columns = ['en']
display(ko_df.head())
display(en_df.head())

,ko
0,하다
1,있다
2,되다
3,나
4,없다


,en
0,the
1,of
2,and
3,to
4,a


In [9]:
ko_df.to_csv('ko_5.8k.csv', index=False)
en_df.to_csv('en_10k.csv', index=False)

In [1]:
import pandas as pd

In [3]:
words_eval = pd.read_csv('koen_word_preprocessed_v1_eval.csv')

In [4]:
words_test = words_eval.sample(frac=0.5, random_state=42).reset_index(drop=True)
words_eval = words_eval.drop(words_test.index).reset_index(drop=True)
display(words_eval.head())
display(words_test.head())

,src_lang,ko,en,fixed
0,en,어쿠스틱,acoustic,False
1,en,선출,elected,False
2,en,보호,protective,False
3,en,결정,determine,False
4,en,렌탈,rentals,False


,src_lang,ko,en,fixed
0,en,범죄자,criminal,False
1,en,눈,eyes,False
2,en,병아리,chick,False
3,en,온다,comes,False
4,en,전통적인,traditional,False


In [5]:
words_eval.to_csv('koen_word_preprocessed_v1_eval.csv', index=False)
words_test.to_csv('koen_word_preprocessed_v1_test.csv', index=False)

In [6]:
import pandas as pd
from datasets import load_dataset
import numpy as np

In [3]:
# sparta = load_dataset('traintogpb/aihub-flores-koen-integrated-sparta-mini-300k')
words = load_dataset('traintogpb/koen-frequent-words-3.6k')

In [7]:
# sparta_train, sparta_eval, sparta_test = pd.DataFrame(sparta['train']), pd.DataFrame(sparta['validation']), pd.DataFrame(sparta['test'])
words_train, words_eval, words_test = pd.DataFrame(words['train']), pd.DataFrame(words['validation']), pd.DataFrame(words['test'])

In [8]:
words_eval.columns

Index(['src_lang', 'ko', 'en', 'fixed'], dtype='object')

In [21]:
words_source = []
for _, row in words_train.iterrows():
    if row['src_lang'] == 'ko':
        words_source.append('words-ko')
    elif row['src_lang'] == 'en':
        words_source.append('words-en')

words_train['src_lang'] = words_source
words_train['ko_ref_xcomet'] = [np.nan] * len(words_train)
words_train.rename(columns={'src_lang': 'source', 'ko': 'ko_ref', 'en': 'en_ref'}, inplace=True)
words_train = words_train[['ko_ref_xcomet', 'ko_ref', 'en_ref', 'source']]
display(words_train.head())

,ko_ref_xcomet,ko_ref,en_ref,source
0,NaN,명령,Command,words-en
1,NaN,넘다,Over,words-en
2,NaN,쉽게,readily,words-en
3,NaN,담배,cigarettes,words-en
4,NaN,우선,First of all,words-en
